### Circle における学生の乗車 Group 分け問題

#### 課題整理
「学生をどの車に割り当てるかを決定する」ので, **「0-1 整数計画問題」** となる

- 学生の乗車 Group 分け問題
    1. 学生をどの車に割り当てるかを決定する
- 旅行に行くための法規制に関する制約
    2. 乗車人数は定員を超えてはいけない
    3. 運転免許証を持つ学生を１人以上、車に割り当てる
- 懇親を目的とした制約
    4. 各学年の学生を１人以上、車に割り当てる
- Gender Balance を考慮した制約
    5. 男女それぞれ１人以上を車に割り当てる


###### Topick
**0-1 整数計画問題**
変数が 0 か 1 をとる問題